In [19]:
import pandas as pd

import sys
import ujson
import pprint
import numpy as np
from operator import itemgetter
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# For displaying google maps
import gmplot
from IPython.display import IFrame

# Second (better?) google map, only inline so far
import gmaps

# For summary stats
import scipy
from scipy import stats

from influxdb import DataFrameClient
#from influxdb import InfluxDBClient


pp = pprint.PrettyPrinter(indent=4)
%pylab inline

Populating the interactive namespace from numpy and matplotlib
time: 21.2 ms


In [20]:
# Testing auto adding time output to all cells
'''
#To install...
wget https://raw.githubusercontent.com/cpcloud/ipython-autotime/master/autotime.py
#Make available via
jupyter nbextension install /Users/patrickmulrooney/class/notebooks/pjmulroo/DSECapstone/autotime.py
'''
 
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.27 ms


In [21]:
def log_progress(sequence, every=None, size=None):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    label.value = u'{index} / {size}'.format(
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = str(index or '?')


time: 49.1 ms


In [22]:
# Read config from local file
GMAP_API_KEY = None

sys.path.append('/Users/patrickmulrooney/Desktop/')
import capstone_config

time: 5.88 ms


In [23]:
GMAP_API_KEY = capstone_config._GMAP_API_KEY
gmaps.configure(api_key=GMAP_API_KEY)

time: 2.21 ms


In [ ]:
dbname = 'endomondo'
client = DataFrameClient('localhost', 8086, 'root', 'root', dbname)

time: 4.41 ms


In [ ]:
endoHRbike = client.query("select * from \"bike (transport)\"")['bike (transport)']

In [ ]:
endoHRbike.groupby('workoutId').max().groupby('userId').count().sort_values(by='altitude')[-5:]

In [ ]:
transports = {}
for i in endoHRbike.groupby('userId').first().index.values:
    transports[i] = endoHRbike[endoHRbike['userId'] == i].reset_index()

In [ ]:
idx = 2734298

m = gmaps.Map()
heatmap_layer = gmaps.Heatmap(data=[list(_i) for _i in list(transports[idx][['latitude','longitude']].values)])

m.add_layer(heatmap_layer)

heatmap_layer.max_intensity = 10
heatmap_layer.point_radius = 2

t = transports[idx].groupby('workoutId').max()['index']
print plt.hist([t.iloc[_i].dayofweek for _i in range(len(t))])



In [ ]:
m

In [ ]:
idx = 4997910

m = gmaps.Map()
heatmap_layer = gmaps.Heatmap(data=[list(_i) for _i in list(transports[idx][['latitude','longitude']].values)])

m.add_layer(heatmap_layer)

heatmap_layer.max_intensity = 100
heatmap_layer.point_radius = 2

t = transports[idx].groupby('workoutId').max()['index']
print plt.hist([t.iloc[_i].dayofweek for _i in range(len(t))])

In [ ]:
m

In [ ]:
idx = 4446822

m = gmaps.Map()
heatmap_layer = gmaps.Heatmap(data=[list(_i) for _i in list(transports[idx][['latitude','longitude']].values)])

m.add_layer(heatmap_layer)

heatmap_layer.max_intensity = 100
heatmap_layer.point_radius = 2

t = transports[idx].groupby('workoutId').max()['index']
print plt.hist([t.iloc[_i].dayofweek for _i in range(len(t))])

In [ ]:
m

In [ ]:
idx = 3351234

m = gmaps.Map()
heatmap_layer = gmaps.Heatmap(data=[list(_i) for _i in list(transports[idx][['latitude','longitude']].values)])

m.add_layer(heatmap_layer)

heatmap_layer.max_intensity = 100
heatmap_layer.point_radius = 2

In [ ]:
t = transports[idx].groupby('workoutId').max()['index']
plt.hist([t.iloc[_i].dayofweek for _i in range(len(t))], bins=7 )
print 'Day of week'
plt.show()

In [ ]:
t = transports[idx].groupby('workoutId').min()['index']
plt.hist([t.iloc[_i].hour+1 for _i in range(len(t))],bins=24)
print 'Start hour'
plt.show()

In [ ]:
t = transports[idx].groupby('workoutId').max()['index']
plt.hist([t.iloc[_i].hour+1 for _i in range(len(t))],bins=24)
print 'End hour'
plt.show()

In [ ]:
transports[idx].max()

In [ ]:
print "All days combined"
m

In [ ]:
m = gmaps.Map()
heatmap_layer = gmaps.Heatmap(data=[list(_i)[0:2] for _i in list(transports[idx][['latitude','longitude','index']].values) if _i[2].dayofweek == 0])

m.add_layer(heatmap_layer)

heatmap_layer.max_intensity = 100
heatmap_layer.point_radius = 2
print "Sunday"
m

In [ ]:
m = gmaps.Map()
heatmap_layer = gmaps.Heatmap(data=[list(_i)[0:2] for _i in list(transports[idx][['latitude','longitude','index']].values) if _i[2].dayofweek == 1])

m.add_layer(heatmap_layer)

heatmap_layer.max_intensity = 100
heatmap_layer.point_radius = 2
print "Monday"
m

In [ ]:
m = gmaps.Map()
heatmap_layer = gmaps.Heatmap(data=[list(_i)[0:2] for _i in list(transports[idx][['latitude','longitude','index']].values) if _i[2].dayofweek == 2])

m.add_layer(heatmap_layer)

heatmap_layer.max_intensity = 100
heatmap_layer.point_radius = 2
print "Tuesday"
m

In [ ]:
m = gmaps.Map()
heatmap_layer = gmaps.Heatmap(data=[list(_i)[0:2] for _i in list(transports[idx][['latitude','longitude','index']].values) if _i[2].dayofweek == 3])

m.add_layer(heatmap_layer)

heatmap_layer.max_intensity = 100
heatmap_layer.point_radius = 2
print "Wednesday"
m

In [ ]:
m = gmaps.Map()
heatmap_layer = gmaps.Heatmap(data=[list(_i)[0:2] for _i in list(transports[idx][['latitude','longitude','index']].values) if _i[2].dayofweek == 5])

m.add_layer(heatmap_layer)

heatmap_layer.max_intensity = 100
heatmap_layer.point_radius = 2
print "Thursday"
m

In [ ]:
transports[idx]['heart_rate'].hist()